In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [18]:
class NetworkAnomalyDetector:
    def __init__(self, sequence_length=10):
        self.sequence_length = sequence_length
        self.scaler = StandardScaler()
        self.model = None
        self.feature_columns = None
        
    def prepare_sequences(self, data):
        """Create sequences for LSTM input"""
        sequences = []
        labels = []
        
        for i in range(len(data) - self.sequence_length):
            sequences.append(data[i:(i + self.sequence_length)])
            labels.append(data.iloc[i + self.sequence_length]['Anomaly'])
            
        return np.array(sequences), np.array(labels)
    
    def preprocess_data(self, df):
        """Preprocess the data and create sequences"""
        # Select numerical features only
        numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
        self.feature_columns = [col for col in numerical_columns if col != 'Anomaly']
        
        # Scale the features
        scaled_data = pd.DataFrame(
            self.scaler.fit_transform(df[self.feature_columns]),
            columns=self.feature_columns
        )
        scaled_data['Anomaly'] = df['Anomaly']
        
        # Create sequences
        X, y = self.prepare_sequences(scaled_data)
        return train_test_split(X, y, test_size=0.2, random_state=42)
    
    def build_model(self, input_shape):
        """Build LSTM model"""
        model = Sequential([
            LSTM(64, activation='relu', input_shape=input_shape, return_sequences=True),
            Dropout(0.2),
            LSTM(32, activation='relu'),
            Dropout(0.2),
            Dense(16, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
        )
        
        model.summary()
        
        self.model = model
        return model
    
    def train(self, df, epochs=50, batch_size=32):
        """Train the model"""
        # Preprocess data
        X_train, X_test, y_train, y_test = self.preprocess_data(df)
        
        # Build model if not already built
        if self.model is None:
            self.build_model((X_train.shape[1], X_train.shape[2]))
        
        # Train model
        history = self.model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_test, y_test),
            verbose=1
        )
        
        self.model.save('model7.h5')
        
        return history, (X_test, y_test)

In [19]:


detector = NetworkAnomalyDetector(sequence_length=10)
df= pd.read_csv("New_dataset2.csv")

In [20]:
# Train the model
history, (X_test, y_test) = detector.train(df, epochs=30)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 10, 64)            25088     
                                                                 
 dropout_6 (Dropout)         (None, 10, 64)            0         
                                                                 
 lstm_7 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_7 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 38049 (148.63 KB)
Trainable params: 3804

C:\Users\ANANT TIWARI\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
